# MNQ Range Program 

by Wolfrank Guzman GitHub: @guzmanwolfrank

The goal of this program is to calculate the first 45 min Open Range on MNQ Futures along with the Day Range and Day Change.  

By creating this dataframe, we can use it to analyze correlations between positive and negative days and their prospective ranges, and the change in these price swings and ranges. 
We can later apply machine learning predictive models to the data and later backtest, deploy live and see if the correlations and projections hold true. 



In [5]:
import os
print (os.path.abspath("testfile.ext"))


C:\Users\Wolfrank\AppData\Local\Programs\Microsoft VS Code\testfile.ext


In [6]:
import yfinance as yf
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Set the ticker symbol 
ticker_symbol = "MNQ=F"
# Fetch intraday data using yfinance for the last 60 days with 15-minute intervals
data = yf.download(ticker_symbol, period="60d", interval="15m").round(2)

# Round numbers
data = data.round(2)
#display(data)
# Convert the index to datetime format
data.index = pd.to_datetime(data.index)

# Filter rows for the specific times 09:30 and 10:15
filtered_data = data[(data.index.time == pd.to_datetime("09:30:00").time()) | (data.index.time == pd.to_datetime("10:15:00").time())]
#display(filtered_data)

# Group by date and aggregate the data
combined_data = filtered_data.groupby(filtered_data.index.date).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

# Calculate the range and add it as a new column
combined_data['Range'] = combined_data['High'] - combined_data['Low']

# Round numbers
data = data.round(2)
cdata = combined_data.round(2)
#display(cdata)

# Calculate the average range for different windows of days
windows = [5, 10, 20, 30, 40, 50, 60]
for window in windows:
    avg_range = combined_data['Range'].rolling(window=window).mean().iloc[-1].round(2)
   # print(f"Average Opening Range for last {window} days:", avg_range)

# Calculate the average of the 'Range' column
average_range = combined_data['Range'].mean()

Ddata = yf.download(ticker_symbol, period="60d", interval="1d").round(2)
Ddata['ABSDayChange']= abs(Ddata['Close']- Ddata['Open'])
Ddata['DayChange']= (Ddata['Close']- Ddata['Open'])
Ddata['OpenOnlyRange']= combined_data['Range']
Ddata['DayRange']= Ddata['High']-Ddata['Low']

# Calculate absolute day change
Ddata['ABSDayChange'] = abs(Ddata['Close'] - Ddata['Open'])

# Calculate day change
Ddata['DayChange'] = Ddata['Close'] - Ddata['Open']

# Calculate OpenOnlyRange
Ddata['OpenOnlyRange'] = combined_data['Range']

# Calculate DayRange
Ddata['DayRange'] = Ddata['High'] - Ddata['Low']

# Define the window sizes
windows = [5, 10, 20, 30, 40, 50]


print("Average Open Range:", average_range)


# Calculate rolling averages for each column and window size
for column in ['ABSDayChange', 'OpenOnlyRange', 'DayRange']:
    print(f"Rolling Averages for {column}:")
    for window in windows:
        rolling_avg = Ddata[column].rolling(window=window).mean().iloc[-1]
        print(f" - {window}-day Average: {rolling_avg:.2f}")
    print()
    

display(Ddata)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Average Open Range: 108.72
Rolling Averages for ABSDayChange:
 - 5-day Average: 142.35
 - 10-day Average: 166.97
 - 20-day Average: 146.41
 - 30-day Average: 137.27
 - 40-day Average: 128.57
 - 50-day Average: 126.88

Rolling Averages for OpenOnlyRange:
 - 5-day Average: 167.65
 - 10-day Average: 143.12
 - 20-day Average: 117.41
 - 30-day Average: 117.86
 - 40-day Average: 109.29
 - 50-day Average: 108.72

Rolling Averages for DayRange:
 - 5-day Average: 257.10
 - 10-day Average: 295.20
 - 20-day Average: 270.50
 - 30-day Average: 257.52
 - 40-day Average: 253.18
 - 50-day Average: 256.90



,Open,High,Low,Close,Adj Close,Volume,ABSDayChange,DayChange,OpenOnlyRange,DayRange
Date,,,,,,,,,,
2024-01-05,16454.00,16577.00,16334.25,16460.25,16460.25,1098485,6.25,6.25,113.25,242.75
2024-01-08,16472.25,16811.50,16378.25,16803.75,16803.75,880010,331.50,331.50,139.00,433.25
2024-01-09,16790.00,16868.50,16658.50,16830.25,16830.25,1023133,40.25,40.25,86.00,210.00
2024-01-10,16829.50,16983.25,16801.75,16945.25,16945.25,986991,115.75,115.75,38.00,181.50
2024-01-11,16955.50,17056.00,16753.00,16966.25,16966.25,1370416,10.75,10.75,212.50,303.00
2024-01-12,16958.00,17042.50,16866.25,16969.25,16969.25,1136604,11.25,11.25,103.75,176.25
2024-01-16,16962.00,17034.00,16812.00,16966.50,16966.50,1590564,4.50,4.50,133.50,222.00
2024-01-17,16979.50,16981.25,16689.00,16869.75,16869.75,1232753,109.75,-109.75,126.25,292.25
2024-01-18,16859.50,17126.00,16833.75,17110.00,17110.00,1383841,250.50,250.50,59.25,292.25
